In [76]:
import json
import random
import logging
import spacy
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.training import Example
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score

In [77]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None


In [78]:
def clean_annotations(train_data):
    cleaned_data = []
    for text, annotation in train_data:
        entities = annotation['entities']
        cleaned_entities = []
        last_end = -1

        for start, end, label in sorted(entities, key=lambda x: x[0]):
            if start >= last_end:  # Ensure there is no overlap
                cleaned_entities.append((start, end, label))
                last_end = end
            else:
                # Optionally, handle overlaps by merging or adjusting entities
                print(f"Found overlapping entity: {(start, end, label)} in text: '{text}'")
                # Merge entities if desired
                # Example: cleaned_entities[-1] = (cleaned_entities[-1][0], max(cleaned_entities[-1][1], end), label)

        cleaned_data.append((text, {"entities": cleaned_entities}))
    return cleaned_data

In [79]:

def train_spacy():

    TRAIN_DATA = convert_dataturks_to_spacy("traindata.json")
    TRAIN_DATA = clean_annotations(TRAIN_DATA)
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe("ner", last=True)


    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(1):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                try:
                  nlp.update(
                      [example],  # batch of texts
                      drop=0.2,  # dropout - make it harder to memorise data
                      sgd=optimizer,  # callable to update weights
                      losses=losses)
                except Exception as error:
                    print(error)
                    continue
            print(losses)
    #test the model and evaluate it
    TEST_DATA = convert_dataturks_to_spacy("testdata.json")
    c=0
    for text, annot in TEST_DATA:
        with open("resume" + str(c) + ".txt", "w") as f:
            doc_to_test = nlp(text)
            d = {}
            for ent in doc_to_test.ents:
                d[ent.label_] = []
            for ent in doc_to_test.ents:
                d[ent.label_].append(ent.text)

            y_true = []
            y_pred = []
            for ent in doc_to_test.ents:
              try:
                doc_gold_text = nlp.make_doc(text)
                gold = Example.from_dict(doc_gold_text, annot)
                y_true.extend([x.ent_type_ if x.ent_type_ else "Not " + ent.label_ for x in gold.reference])
                y_pred.extend([x.ent_type_ if x.ent_type_ else "Not " + ent.label_ for x in doc_to_test])
              except Exception as e:
                pass
            try:
                print("\nFor Entity " + ent.label_ + ":\n")
                print("Classification Report:\n", classification_report(y_true, y_pred))
                p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
                a = accuracy_score(y_true, y_pred)
                print("Accuracy:", a)
                print("Precision:", p)
                print("Recall:", r)
                print("F-score:", f)
            except Exception as e:
                print(e)
                pass

            c += 1
    print("Accuracy:", a)
    print("Precision:", p)
    print("Recall:", r)
    print("F-score:", f)

In [85]:
train_spacy()

Streaming output truncated to the last 5000 lines.
• knowledge of SLP Multimedia Framework
• Ported Player (Audio and Video) module
• Ported Audio Out and Audio In modules using ALSA library
• ETMs Firmware Client: Wabtec Railway Electronics, German Town

Syam Prasad Devendla
Environment: Windows XP, IAR Workbench Hardware: IOC board (Provided by WRE)
Languages: C
• Serial - Ethernet Bridge
• Serial communication commands handling.
• Supt Link
Client: Schindler Elevator & Escalator Corporation
Environment: Windows Mobile 5.0, embedded VC, Visual Studio 2005
Languages: VC++, MFC
• A Business application. It makes easy for the superintendents to check the status of the
elevators and escalators which are newly installed or being maintained and rate them
according to their performance and report the same to the Sap server using mobility
solutions.
• FldLink
Client: Schindler Elevator & Escalator Corporation
Environment: Windows Mobile 5.0, embedded VC, Visual Studio 2005
Languages: VC++, M

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Yogi Pesaru
Developer - Infosys Limited

Hyderabad..." with entities "[(0, 11, 'Name'), (12, 21, 'Designation'), (24, 40...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Palani S
Senior Technology Support Executive at In..." with entities "[(0, 8, 'Name'), (9, 44, 'Designation'), (47, 55, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

[E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated entity spans have leading or trailing whitespace or punctuation. You can also use the `debug data` command to validate your JSON-formatted training data. For details, run:
python -m spacy debug data --help


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Gunjan Nayyar
Hoshiarpur, Punjab - Email me on Ind..." with entities "[(0, 13, 'Name'), (14, 24, 'Location'), (55, 98, '...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Partho Sarathi Mitra
Kolkata, West Bengal - Email ..." with entities "[(0, 20, 'Name'), (21, 28, 'Location'), (63, 115, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

[E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated entity spans have leading or trailing whitespace or punctuation. You can also use the `debug data` command to validate your JSON-formatted training data. For details, run:
python -m spacy debug data --help


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Dinesh Reddy
Deployed chef for configuration manag..." with entities "[(4611, 4621, 'Skills'), (4531, 4567, 'College Nam...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned e


For Entity Degree:

Classification Report:
                      precision    recall  f1-score   support

       College Name       0.00      0.00      0.00        16
Companies worked at       0.00      0.00      0.00        20
             Degree       0.03      1.00      0.06         4
        Designation       0.00      0.00      0.00        88
      Email Address       1.00      1.00      1.00         4
           Location       0.00      0.00      0.00        16
               Name       1.00      1.00      1.00         8
         Not Degree       0.96      0.97      0.97       842
  Not Email Address       0.96      0.97      0.97       842
       Not Location       0.96      0.97      0.97       842
           Not Name       0.96      0.97      0.97       842
             Skills       0.00      0.00      0.00         8

           accuracy                           0.93      3532
          macro avg       0.49      0.57      0.49      3532
       weighted avg       0.92      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


For Entity Graduation Year:

max() arg is an empty sequence

For Entity Graduation Year:

Classification Report:
                      precision    recall  f1-score   support

       College Name       0.00      0.00      0.00         6
      Email Address       0.00      0.00      0.00         0
    Graduation Year       1.00      1.00      1.00         3
           Location       0.00      0.00      0.00        18
               Name       1.00      1.00      1.00         6
  Not Email Address       0.96      1.00      0.98      1183
Not Graduation Year       0.96      1.00      0.98      1183
           Not Name       0.96      1.00      0.98      1183
             Skills       0.00      0.00      0.00       111

           accuracy                           0.96      3693
          macro avg       0.54      0.56      0.55      3693
       weighted avg       0.93      0.96      0.95      3693

Accuracy: 0.9634443541835905
Precision: 0.9289828043142482
Recall: 0.9634443541835905
F-s

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Dushyant Bhatt
BI / Big Data/ Azure

Hyderabad-Dec..." with entities "[(5893, 6043, 'Skills'), (5879, 5883, 'Graduation ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

Accuracy: 0.9593826157595451
Precision: 0.9351834512981235
Recall: 0.9593826157595451
F-score: 0.9456206323750364

Accuracy: 0.9634443541835905
Precision: 0.9329879420039412
Recall: 0.9634443541835905
F-score: 0.9463293162034578

Accuracy: 0.9601949634443542
Precision: 0.9336098866708394
Recall: 0.9601949634443542
F-score: 0.9457312532094156

Accuracy: 0.9634443541835905
Precision: 0.9289828043142482
Recall: 0.9634443541835905
F-score: 0.9458989924866265